# SMT Solver for VLSI

In [1]:
from itertools import combinations
from z3 import *
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import time

In [2]:
def plot_result(dims, w, ys, xs):
    lims = (0, max(w, max([ys[i]+dims[i][1] for i in range(len(ys))])))

    fig1 = plt.figure(figsize=(10, 10))
    ax1 = fig1.add_subplot(111, aspect='equal')
    for i in range(len(xs)):
        ax1.add_patch(patches.Rectangle((xs[i], ys[i]), dims[i][0], dims[i][1], edgecolor='black', facecolor=random.choice(['r', 'g', 'y'])))

    plt.xticks(range(w+1))
    plt.yticks(range(max([ys[i]+dims[i][1] for i in range(len(ys))])+1))
    #plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.show()

In [3]:
def max_z3(vars):
    max = vars[0]
    for v in vars[1:]:
        max = If(v > max, v, max)
    return max

In [4]:
def read_instance(instance_id):
    filepath = "../instances/ins-" + str(instance_id) + ".txt"
    with open(filepath, "r") as f_in:
        f = f_in.readlines()
        for i in range(len(f)):
            if not f[i][-1].isnumeric():
                f[i] = f[i][:-1]
                
        W = int(f[0])
        n = int(f[1])
        dims_line = f[2].split(" ")    
        dims = [[int(dims_line[0]), int(dims_line[1])]]
        for i in range(1, int(f[1])-1):
            dims_line = f[2 + i].split(" ")
            dims.append([int(dims_line[0]), int(dims_line[1])])
        dims_line = f[-1].split(" ")
        dims.append([int(dims_line[0]), int(dims_line[1])])
        
        dims = sorted(dims, key=lambda l:l[0]*l[1], reverse=True)
        
    return dims, W, n

In [5]:
def alphaMEncoding(opt, A, B):
    n = len(A)
    print(A)
    print(B)
    alphas = [Bool("a_%s" % (i)) for i in range(n+1)]
    opt.add(alphas[0] == True)
    T1 = [Bool("t1_%s" % (i)) for i in range(n)]
    T2 = [Bool("t2_%s" % (i)) for i in range(n)]
    T3 = [Bool("t3_%s" % (i)) for i in range(n)]
    for i in range(n):
        opt.add(Implies(T1[i], A[i] <= B[i]))
        opt.add(Implies(A[i] <= B[i], T1[i]))
        opt.add(Implies(T2[i], A[i] < B[i]))
        opt.add(Implies(A[i] < B[i], T2[i]))
        opt.add(Implies(T3[i], Or(T2[i], alphas[i+1])))
        opt.add(Implies(Or(T2[i], alphas[i+1]), T3[i]))
        opt.add(Implies(alphas[i], And(T3[i], T1[i])))
        opt.add(Implies(And(T3[i], T1[i]), alphas[i]))
    return opt

In [ ]:
dims, W, n = read_instance(11)
# Solver
opt = Optimize()

sol = [[Int("x_%s" % (i)), Int("y_%s" % (i))] for i in range(n)]

# overlapping
for i in range(n):
    for j in range(n):
        if i != j:
            opt.add(Or(sol[i][0] + dims[i][0] <= sol[j][0],
                       sol[j][0] + dims[j][0] <= sol[i][0],
                       sol[i][1] + dims[i][1] <= sol[j][1],
                       sol[j][1] + dims[j][1] <= sol[i][1]))

# width
for i in range(n):
    opt.add(sol[i][0] + dims[i][0] <= W)
    opt.add(sol[i][0] >= 0) # min
    opt.add(sol[i][0] <= W - min([dims[i][0] for i in range(n)])) # max
    
# height min
for i in range(n):
    opt.add(sol[i][1] >= 0)

# Symmetry breaking
flags = [False for i in range(n)]
for i in range(n):
    equal_hor = []
    equal_ver = []
    for j in range(n):
        if i<=j and dims[i][0] == dims[j][0] and dims[i][1] == dims[j][1] and not flags[j]:
            equal_hor.append(sol[j][0])
            flags[j] = True
            equal_ver.append(sol[j][1])
            
    if len(equal_hor) > 1:
        opt = alphaMEncoding(opt, equal_hor, [equal_hor[len(equal_hor)-1-k] for k in range(len(equal_hor))])
        opt = alphaMEncoding(opt, equal_ver, [equal_ver[len(equal_ver)-1-k] for k in range(len(equal_ver))])
    
for i in range(n):
    for j in range(n):
        if i<j and dims[i][0] == dims[j][0]:
            opt.add(Implies(sol[i][0]==sol[j][0], sol[i][1]<=sol[j][1]))
        if i<j and dims[i][1] == dims[j][1]:
            opt.add(Implies(sol[i][1]==sol[j][1], sol[i][0]<=sol[j][0]))

height_opt = Int('height_opt')

# Biggest rectangle
opt.add(And(sol[0][0] <= 1+(W-dims[0][0])/2, sol[0][1] <= 1+(height_opt-dims[0][1])/2))

opt.add(height_opt <= 2*max(max([dims[i][1] for i in range(n)]), min_height)) # upper bound
opt.add(height_opt >= int(sum([dims[i][1]*dims[i][0] for i in range(n)])/W)) # lower bound
objective = height_opt == max_z3([sol[i][1] + dims[i][1] for i in range(n)])
opt.add(objective)
opt.minimize(height_opt)

start_time = time.time()
opt.check()
m = opt.model()
print("Solving time (s): ", time.time()-start_time)

ys = []
xs = []
for i in range(n):
    xs.append(int(m.evaluate(sol[i][0]).as_string()))
    ys.append(int(m.evaluate(sol[i][1]).as_string()))
    
plot_result(dims, W, ys, xs)